# Initialization

## Formatter

In [1]:
using JupyterFormatter
enable_autoformat()

1-element Vector{Function}:
 format_current_cell (generic function with 1 method)

## Load Packages

In [2]:
using HalfIntegers
using LinearAlgebra
using ElasticArrays
using JLD2
using Distributed
using DelimitedFiles
using Random
using CSV
using DataFrames
using LsqFit

## Plot style

In [3]:
using Plots
using Plots.PlotMeasures
using LaTeXStrings

# set backend and style
pgfplotsx()
theme(:default)
default(
    markerstrokewidth=0,
    markerstrokealpha=0,
    linewidth=2,
    #grid=:none,   
    gridlinewidth=0.5,
    markersize=5,
    markershape=:circle,
    tickfontsize=18,
    size=(900, 600),
    legendfontsize=18,
    guidefontsize=20,
    titlefontsize=20,
    legend=(0.03, 0.98),
    xticks=0:10,
    foreground_color_axis="black",
    foreground_color_border="black",
    foreground_color_guide="darkorange",
    foreground_color_text="black",
    guidefontcolor="black",
    plot_titlefontcolor="black",
    titlefontcolor="black",
    shape=[:circle],
)

## Data

In [4]:
include("../../parameters.jl");
DATA_FOLDER = "../../data";
STORE_FOLDER = "./plots";

## Useful Function 

In [7]:
function Fromj0ToMass(j0_float)
    round(sqrt(2 * j0_float * immirzi), digits=2)
end

Fromj0ToMass (generic function with 1 method)

In [8]:
function Fromj0Tojpm(j0_float)
    round(Int64, 2 * (j0_float / (sqrt(6)))) / 2
end

Fromj0Tojpm (generic function with 1 method)

# Computational times

In [61]:
workers = 64
threads = 1

comp_times = Matrix(
    DataFrame(
        CSV.File(
            "../../immirzi_$(immirzi)_workers_$(workers)_threads_$(threads).csv",
            header=true,
        ),
    ),
);

In [18]:
pl1 = plot(
    1:11,
    [
        comp_times[:, 1],
        comp_times[:, 2],
        comp_times[:, 3],
        comp_times[:, 4],
        comp_times[:, 5],
        comp_times[:, 6],
        comp_times[:, 7],
        comp_times[:, 8],
        comp_times[:, 9],
    ],
    label=[L"j_0=1,\,j_{\pm}=0.5" L"j_0=1.5,\,j_{\pm}=0.5" L"j_0=2,\,j_{\pm}=1" L"j_0=2.5,\,j_{\pm}=1" L"j_0=3,\,j_{\pm}=1" L"j_0=3.5,\,j_{\pm}=1.5" L"j_0=4,\,j_{\pm}=1.5" L"j_0=4.5,\,j_{\pm}=2" L"j_0=5,\,j_{\pm}=2"],
    markershape=:circle,
    yaxis=:log,
    xaxis=:log,
    legend=(0.10, 0.97),
    legendfontsize=18,
)
xticks!(
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
    string.([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]),
)
xlabel!(L"\Delta l")
ylabel!(L"\textrm{seconds}")
title!(L"\textrm{Computation time of EPRL vertices}")
savefig(
    "./plots/narval_computation_workers_$(workers)_threads_$(threads)_immirzi_$(immirzi).pdf",
)

"/home/frisus/Scrivania/BlackToWhiteHole/notebooks/Julia/plots/narval_computation_workers_64_threads_1_immirzi_5.pdf"

# Amplitude

In [5]:
function AmplitudeAbsSquaredLoad(
    DATA_FOLDER,
    j0,
    jpm,
    alpha,
    immirzi,
    T,
    K0=0.5,
    Kpm=0.5)

    if j0 > 5
        K0 = 0.0
        Kpm = 0.0
    end

    Matrix(
        DataFrame(
            CSV.File(
                "$(DATA_FOLDER)/amplitude_data/j0=$(j0)_jpm=$(jpm)/K0_$(K0)_Kpm_$(Kpm)/immirzi_$(immirzi)/alpha_$(alpha)/amplitude_abs_sq_T_$(T).csv",
                header=true,
            ),
        ),
    )

end

function AmplitudeRescaling(ampls, rescaling_factor=100000000000000000000000000000000000000000000000000)
    ampls[:] .= ampls[:] .* rescaling_factor
end

function TRange(m, immirzi, T_sampling_parameter)
    LinRange(0, 4 * pi * m / immirzi, T_sampling_parameter)
end

TRange (generic function with 1 method)

In [25]:
ampls_path = "$(STORE_FOLDER)/ampls/T_sampling_parameter_$(T_sampling_parameter)"
mkpath("$(ampls_path)")

Dl_range = Dl_max - Dl_min + 1

ampls_collection = Array{Float64,3}(undef, T_sampling_parameter, Dl_range, length(angular_spins))

Lifetime_x_coordinates = Float64[]

counter = 0
for user_conf in angular_spins

    counter += 1
    j0_float = user_conf[1]
    j0 = half(2 * j0_float)

    if length(user_conf) == 1
        jpm_float = round(Int64, 2 * (j0_float / (sqrt(6)))) / 2
        jpm = half(2 * jpm_float)
    else
        jpm_float = user_conf[2]
        jpm = half(2 * jpm_float)
    end

    m = Fromj0ToMass(j0)
    T_range = TRange(m, immirzi, T_sampling_parameter)
    push!(Lifetime_x_coordinates, m)

    DeltaT = Int(T_sampling_parameter / 5)
    X_ticks = [0:DeltaT:T_sampling_parameter;]
    X_labels = String[]

    for x in X_ticks[1:end-1]
        l = T_range[x+1]
        l = round(l, digits=2)
        push!(X_labels, "$(l)")
    end

    final_x_tick = T_range[end]
    push!(X_labels, L"\frac{4 \pi m}{\gamma}")

    ampls = AmplitudeAbsSquaredLoad(DATA_FOLDER, j0_float, jpm_float, alpha, immirzi, T_sampling_parameter)
    AmplitudeRescaling(ampls)
    ampls_collection[:, :, counter] .= ampls[:, :]

    pl1 = plot(
        1:T_sampling_parameter,
        [ampls[:, 1] ampls[:, 2] ampls[:, 3] ampls[:, 4] ampls[:, 5] ampls[:, 6] ampls[:, 7] ampls[:, 8] ampls[:, 9]], #ampls[:, 10] ampls[:, 11]],
        label=["Δl=0" "Δl=1" "Δl=2" "Δl=3" "Δl=4" "Δl=5" "Δl=6" "Δl=7" "Δl=8"],# "Δl=9" "Δl=10"],
        legend=(0.35, 0.90),
        #label = "",
        markershape=:circle,
        grid=:true,
        xlims=(0, T_sampling_parameter + 1),
        xticks=(X_ticks, X_labels),
        palette=palette([:skyblue, :purple], Dl_range),
        legendfontsize=20)
    xlabel!(L"T")
    title!(L"|W_{\alpha=%$alpha}  \, \left(m=%$m, \, T \right) |^2 \times 10^{50}")
    savefig("$(ampls_path)/W_alpha_$(alpha)_Immirzi_$(immirzi)_j0_$(j0_float).pdf")

end

In [29]:
#Y_ticks = [exp(-60):1.0;]
#Y_labels = [L"e^{-60}",L"e^{-30}",L"1"]

pl1 = plot(
    Lifetime_x_coordinates,
    [ampls_collection[end, 1, :] ampls_collection[end, 2, :] ampls_collection[end, 3, :] ampls_collection[end, 4, :] ampls_collection[end, 5, :] ampls_collection[end, 6, :] ampls_collection[end, 7, :] ampls_collection[end, 8, :] ampls_collection[end, 9, :]],# ampls_collection[100, 10, :] ampls_collection[100, 11, :]],
    label=["Δl=0" "Δl=1" "Δl=2" "Δl=3" "Δl=4" "Δl=5" "Δl=6" "Δl=7" "Δl=8"], #"Δl=9" "Δl=10"],
    legend=(0.10, 0.60),
    #label = "",
    markershape=:circle,
    grid=:true,
    #xscale = :log,
    yscale=:ln,
    #xlims=(0, T_sampling_parameter + 1),
    #xticks=(X_ticks, X_labels),
    #yticks=(Y_ticks, Y_labels),
    palette=palette([:skyblue, :purple], 9),
    legendfontsize=20)
    xlabel!(L"m")
    title!(L"\mu \left( m, \, T \right) |W_{\alpha}  \, \left(m, \, T \right) |^2")
    savefig("$(ampls_path)/Lifetime_alpha_$(alpha)_Immirzi_$(immirzi).pdf")

"/home/frisus/Scrivania/BlackToWhiteHole/notebooks/Julia/plots/ampls/T_sampling_parameter_100/Lifetime_alpha_3_Immirzi_5.pdf"

In [30]:
Lifetime_x_coordinates

19-element Vector{Float64}:
  3.16
  3.87
  4.47
  5.0
  5.48
  5.92
  6.32
  6.71
  7.07
  7.42
  7.75
  8.06
  8.37
  8.66
  8.94
  9.22
  9.49
  9.75
 10.0

# Crossing Time

In [113]:
function CrossingTimeLoad(DATA_FOLDER, j0, jpm, alpha, immirzi, T, K0=0.5, Kpm=0.5)
    return Matrix(
        DataFrame(
            CSV.File(
                "$(DATA_FOLDER)/amplitude_data/j0=$(j0)_jpm=$(jpm)/K0_$(K0)_Kpm_$(Kpm)/immirzi_$(immirzi)/alpha_$(alpha)/crossing_time_$(T).csv",
                header=true,
                # si chiamava "lifetime" prima (boh)
            ),
        ),
    )
end

CrossingTimeLoad (generic function with 3 methods)

In [114]:
mkpath("$(STORE_FOLDER)/crossing_times")

crossing_times = Float64[]
mass_values = Float64[]
mass_values_string = String[]
n_points = size(angular_spins)[1]
semiclassical_estimate = Float64[]

for j0_index in angular_spins[1:end]

    j0 = j0_index[1]
    jpm = Fromj0Tojpm(j0)
    m = Fromj0ToMass(j0)

    crossing_time =
        CrossingTimeLoad(DATA_FOLDER, j0, jpm, alpha, immirzi, T_sampling_parameter)

    push!(crossing_times, crossing_time[end])
    push!(mass_values, m)
    push!(mass_values_string, "$(m)")
    push!(semiclassical_estimate, 2 * m * pi / immirzi)

end

In [115]:
crossing_times_alpha_6 = crossing_times

9-element Vector{Float64}:
  8.552356894652252
  6.715555011558749
  8.885764069218283
  9.934588265796084
 10.882796185405248
 11.754763358538893
 12.566370614359158
 13.328648814475041
 14.049629462081306

In [116]:
pl1 = scatter(
    mass_values[2:end],
    [[crossing_times_alpha_3[2:end] crossing_times_alpha_4[2:end] crossing_times_alpha_5[2:end] crossing_times_alpha_6[2:end]]],
    label=[L"\alpha=3" L"\alpha=4" L"\alpha=5" L"\alpha=6"],
    legend=(0.10, 0.97),
    markersize=6,
    linewidth=-1,
    markershape=:circle,
    #linestyle = :none,
    legendfontsize=18,
    xticks=mass_values,
)
xlabel!(L"m")
title!(L"\tau \left(m \right)")

plot!(
    mass_values[2:end],
    [[semiclassical_estimate[2:end]]],
    label="semicl.",
    linestyle=:dash,
    markershape=:none,
)

savefig("$(STORE_FOLDER)/crossing_times/CT_Immirzi_$(immirzi)_T_$(T_sampling_parameter).pdf")

"/home/frisus/Scrivania/BlackToWhiteHole/notebooks/Julia/plots/crossing_times/CT_Immirzi_1_T_100.pdf"

# Lifetime